# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846780565437                   -0.70    4.5
  2   -7.852307089080       -2.26       -1.53    1.0
  3   -7.852613568499       -3.51       -2.55    1.5
  4   -7.852645805798       -4.49       -2.86    2.8
  5   -7.852646459403       -6.18       -3.14    1.0
  6   -7.852646677780       -6.66       -4.11    1.0
  7   -7.852646686579       -8.06       -4.88    2.0
  8   -7.852646686723       -9.84       -5.53    1.5
  9   -7.852646686729      -11.21       -5.93    1.5
 10   -7.852646686730      -12.23       -6.96    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846872003394                   -0.70           4.5
  2   -7.852527932448       -2.25       -1.63   0.80    2.0
  3   -7.852637413508       -3.96       -2.71   0.80    1.0
  4   -7.852646445346       -5.04       -3.30   0.80    2.0
  5   -7.852646667008       -6.65       -4.09   0.80    1.0
  6   -7.852646686458       -7.71       -4.66   0.80    2.2
  7   -7.852646686726       -9.57       -5.46   0.80    1.5
  8   -7.852646686730      -11.46       -6.73   0.80    1.8


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.393172e+01     3.489963e+00
 * time: 0.4359889030456543
     1     1.006765e+00     1.787797e+00
 * time: 0.63714599609375
     2    -1.218206e+00     2.075597e+00
 * time: 0.6638948917388916
     3    -3.599640e+00     1.886257e+00
 * time: 0.7020909786224365
     4    -4.475754e+00     1.806459e+00
 * time: 0.7403469085693359
     5    -6.446152e+00     1.296440e+00
 * time: 0.7784478664398193
     6    -7.197312e+00     8.198472e-01
 * time: 0.8168320655822754
     7    -7.475116e+00     8.242275e-01
 * time: 0.8433098793029785
     8    -7.692006e+00     5.061305e-01
 * time: 0.869891881942749
     9    -7.775580e+00     3.006914e-01
 * time: 0.8963210582733154
    10    -7.812844e+00     8.452223e-02
 * time: 0.922882080078125
    11    -7.830136e+00     1.475323e-01
 * time: 0.9494779109954834
    12    -7.840010e+00     8.966894e-02
 * time: 0.9761040210723877
    13    -7.848751e+00     2.956687e-02
 * time: 1.00268006324768

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846861438340                   -0.70    4.8


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645890319                   -1.64
  2   -7.852646686730       -6.10       -3.70
  3   -7.852646686730      -13.29       -7.24


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.3327423659784e-8
|ρ_newton - ρ_scfv| = 1.6238220812091434e-7
|ρ_newton - ρ_dm|   = 2.346630041340683e-9
